In [10]:
#!pip install ffmpeg-python>=0.2.0
#!pip install matplotlib>=3.0.2
#!pip install munkres>=1.1.2
#!pip install numpy>=1.16
#!pip install opencv-python>=3.4
#!pip install Pillow>=5.4
#!pip install vidgear>=0.1.4
#!pip install torch>=1.4.0
#!pip install torchvision>=0.5.0
#!pip install tqdm>=4.26
#!pip install tensorboard>=1.11
#!pip install tensorboardX>=1.4
#!pip install backports.entry-points-selectable
#!pip install backports.tempfile
#!pip uninstall -y setuptools
#!pip install setuptools==65.5.0
#!pip install fastdtw

### Imports

In [11]:
# Essential Imports
import cv2
import numpy as np
import os
import time
import torch
import pathlib
import math
import re
import pandas as pd
import matplotlib.pyplot as plt
from SimpleHRNet import SimpleHRNet
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from scipy.signal import find_peaks
import scipy.ndimage

### Constants & Configuration

This cell defines all constants, file paths, processing parameters and exercise-specific configurations.

* **File and Model Paths:**
    * `Model_path`: (`pathlib.Path`) - The full path to downloaded SimpleHRNet model weights
    * `Video_baseline_path`: (`pathlib.Path`) - The full path to the main folder containing subfolders for each baseline exercise video (filenames should include angle and rep count (`squat_10reps_front_video1.mp4`).
    * `Video_test_path`: (`pathlib.Path`) - The full path to the test video file
* **Model & Pose Structure:**
    * `Expected_Joint_Count`: (`int`, Unit = Count) - The number of keypoints the HRNet model detects (17 for COCO).
    * `Jointnames_list`: (`list`, Unit = Name) - List of names corresponding to the keypoint indices output by the model (must match model output)

* **Processing Settings:**
* `Target_rep_length`: (`int`, Unit = Frames / Timesteps). This parameter defines a standard length (number of data points or "frames") for every single repetition segment after it has been detected because exercise repetitions can vary in duration. To compare them using DTW, they need to be represented on the same timescale. The `time_normalize_sequence` function takes a repetition segment and resamples it using linear interpolation to produce a new sequence that always has exactly the same number of frames as `Target_rep_length`. By normalizing all repetitions we ensure that the DTW distances reflect differences in the shape of the movement over that standard duration.
  
* **User Settings:**
    * `DTW_Distance_threshold`: (`float`, Unit = Unitless Distance) - Used if a specific derived threshold isn't available. Lower values are stricter for matching
    * `Tuning_Prominance_range`: (`list` of `float`, Unit = Unitless Ratio) - Defines the search space for the `peak_prominence` parameter during automatic tuning of repetition counting.
    * `Tuning_Distance_range`: (`list` of `int`, Unit = Frames) - Search space for `peak_distance` parameter during automatic tuning. Controls the minimum number of frames separating detected repetitions
    * `Smoothing_window_size`: (`int`, Unit = Frames) - Size of moving average window to smooth trajectory signal before peak detection (reduce noise)
    * `Default_Peak_Prominence`: (`float`, Unit = Unitless Ratio) - Prominence value used for repetition counting if tuning fails
    * `Default_Peak_Distance`: (`int`, Unit = Frames) - Minimum distance between peaks used for repetition counting if tuning fails
    * `Threshold_STD_Multiplier`: (`float`, Unit = Unitless) - Factor (N) used when deriving thresholds automatically (Mean + N * StdDev)
    * `Min_REPS_to_calc_threshold`: (`int`, Unit = Count) - Min. number of valid repetitions within a specific baseline angle to derive a threshold.

In [12]:
Model_path = pathlib.Path("C:/Users/michel.marien_icarew/Documents/Privé/Opleiding/Mastervakken/Deep Neural Networks/Opdacht 2/simple-HRNet/weights/pose_hrnet_w48_256x192.pth") # Path to HRNet weights
Video_baseline_path = "C:/Users/michel.marien_icarew/Documents/Privé/Opleiding/Mastervakken/Deep Neural Networks/Opdacht 2/baselines2"
Video_test_path = "C:/Users/michel.marien_icarew/Documents/Privé/Opleiding/Mastervakken/Deep Neural Networks/Opdacht 2/test_videos/test_excercise.mp4" # Video file to analyze

# load pre-trained data
Use_precalculated_parameters = True # True to load from file, False to recreate baseline
Precomputed_model_parameters = "exercise_learned_summary.xlsx" # File with parameters

# Model & Pose Settings
Expected_Joint_Count = 17 # COCO keypoints format
Jointnames_list = ["Nose","Left Eye","Right Eye","Left Ear","Right Ear","Left Shoulder", "Right Shoulder","Left Elbow","Right Elbow","Left Wrist",
                    "Right Wrist", "Left Hip","Right Hip","Left Knee","Right Knee","Left Ankle","Right Ankle"]

# Recognition & Processing Parameters
Target_rep_length = 100
DTW_Distance_threshold = 500.0

# Parameters for Repetition Counting Tuning (Grid Search Ranges)
Tuning_Prominance_range = [0.02, 0.04, 0.06, 0.08, 0.10, 0.12, 0.14]
Tuning_Distance_range = [5, 8, 10, 12, 15, 20, 25, 30, 35]

# Parameters for Threshold Derivation
Threshold_STD_Multiplier = 2.0 # Default for mean + N*stddev calculation
Min_REPS_to_calc_threshold = 1     # Min reps needed in baseline angle category to derive threshold

# Fallback if tuning fails or not enough data
Smoothing_window_size = 5
Default_Peak_Prominence = 0.05
Default_Peak_Distance = 10


#### Configuration Block: `EXERCISE_CONFIG`

This dictionary (`EXERCISE_CONFIG`) defines the movement trajectory for each specific exercise during the repetition counting steps. The keys of the dictionary (`'tricep_pushdown'`) should match the corresponding baseline subfolder names (pay attention to lowercase/underscores).

Running this code block defines the `EXERCISE_CONFIG` variable in the notebook's memory.

* `EXERCISE_CONFIG`: (`dict`)
    * Keys: Exercise label strings (`'squat'`).
    * Values: A nested dictionary containing configuration for that specific exercise. Currently, it holds the `'trajectory_logic'`.

For each exercise, the nested `'trajectory_logic'` dictionary specifies:

* `'metric'`: (`str`)
    * Defines what measurement represents the cyclical motion of the repetition (`'elbow_angle'`, `'hip_y'`, `'shoulder_y'`).
    * Unit = Degrees for angles, normalized Y-coordinate units for `_y` metrics).
* `'joints'`: (`list` of `str`)
    * Specifies the base names of the joints required to calculate the chosen `metric`. For metrics involving paired joints (like angles or averaging Y-coordinates), the code will automatically look for 'Left' and 'Right' versions (if 'Shoulder' is listed for a `_y` metric, it will try to find 'Left Shoulder' and 'Right Shoulder'). For angle metrics needing 3 joints, list them in order (`['Shoulder', 'Elbow', 'Wrist']`).
    * Unit = Joint names (strings).
* `'invert_for_valley'`: (`bool`)
    * Tells the peak-finding algorithm if the moment of a rep. cycle corresponds to a min. value (`True`) or a max.m value (`False`) in the calculated `trajectory` signal (bottom of a squat, lowest point of a dip, most extended arm in a bicep curl).
    * Unit = Boolean.

In [13]:
EXERCISE_CONFIG = {
    'tricep_pushdown': {
        'trajectory_logic': {
            'metric': 'elbow_angle', # Use 'elbow_angle' # also possible: 'shoulder_y', 'hip_y', 'wrist_y', 'knee_angle', etc
            'joints': ['Shoulder', 'Elbow', 'Wrist'], # Base joint names
            'invert_for_valley': False # Peak angle marks end of repetition
        },
    },
    'tricep_dips': {
        'trajectory_logic': {
            'metric': 'shoulder_y', # Use Y-coordinate of shoulders # also possible: 'shoulder_y', 'hip_y', 'wrist_y', 'knee_angle', etc
            'joints': ['Shoulder'], # Base joint names
            'invert_for_valley': True # Lowest point (valley) marks end of repetition
        },
    },
    'squat': {
         'trajectory_logic': {
             'metric': 'knee_angle', # Use Y-coordinate of hips # also possible: 'shoulder_y', 'hip_y', 'wrist_y', 'knee_angle', etc
             'joints': ['Hip', 'Knee', 'Ankle'], # Base joint names
             'invert_for_valley': True # Lowest point (valley) marks end of repetition
         },
    },
    'barbell_biceps': {
        'trajectory_logic': {
            'metric': 'wrist_y', # Use Y-coordinate of hips # also possible: 'shoulder_y', 'hip_y', 'wrist_y', 'knee_angle', etc
            'joints': ['Shoulder', 'Elbow', 'Wrist'], # Base joint names
            'invert_for_valley': False # Lowest point (valley) marks end of repetition
         },
    },
    'bench_press': {
        'trajectory_logic': {
            'metric': 'elbow_y', # Use Y-coordinate of hips # also possible: 'shoulder_y', 'hip_y', 'wrist_y', 'knee_angle', etc
            'joints': ['Elbow'], # Base joint names
            'invert_for_valley': False # Lowest point (valley) marks end of repetition
         },
    },
    'chest_fly_machine': {
        'trajectory_logic': {
            'metric': 'wrist_x', # Use Y-coordinate of hips # also possible: 'shoulder_y', 'hip_y', 'wrist_y', 'knee_angle', etc
            'joints': ['Wrist'], # Base joint names
            'invert_for_valley': True # Lowest point (valley) marks end of repetition
         },
    },
    'deadlift': {
        'trajectory_logic': {
            'metric': 'shoulder_y', # Use Y-coordinate of hips # also possible: 'shoulder_y', 'hip_y', 'wrist_y', 'knee_angle', etc
            'joints': ['Shoulder', 'Hip', 'Knee'], # Base joint names
            'invert_for_valley': True # Lowest point (valley) marks end of repetition
         },
    },
    'decline_bench_press': {
        'trajectory_logic': {
            'metric': 'shoulder_angle', # Use Y-coordinate of hips # also possible: 'shoulder_y', 'hip_y', 'wrist_y', 'knee_angle', etc
            'joints': ['Shoulder', 'Elbow', 'Wrist'], # Base joint names
            'invert_for_valley': False # Lowest point (valley) marks end of repetition
         },
    },
    'hammer_curl': {
        'trajectory_logic': {
            'metric': 'wrist_y', # Use Y-coordinate of hips # also possible: 'shoulder_y', 'hip_y', 'wrist_y', 'knee_angle', etc
            'joints': ['Wrist'], # Base joint names
            'invert_for_valley': False # Lowest point (valley) marks end of repetition
         },
    },
    'hip_thrust': {
        'trajectory_logic': {
            'metric': 'hip_y', # Use Y-coordinate of hips # also possible: 'shoulder_y', 'hip_y', 'wrist_y', 'knee_angle', etc
            'joints': ['Hip'], # Base joint names
            'invert_for_valley': True # Lowest point (valley) marks end of repetition
         },
    },
    'incline_bench_press': {
        'trajectory_logic': {
            'metric': 'elbow_angle', # Use Y-coordinate of hips # also possible: 'shoulder_y', 'hip_y', 'wrist_y', 'knee_angle', etc
            'joints': ['Wrist', 'Elbow', 'Shoulder'], # Base joint names
            'invert_for_valley': False # Lowest point (valley) marks end of repetition
         },
    },
    'lat_pulldown': {
        'trajectory_logic': {
            'metric': 'wrist_y', # Use Y-coordinate of hips # also possible: 'shoulder_y', 'hip_y', 'wrist_y', 'knee_angle', etc
            'joints': ['Wrist'], # Base joint names
            'invert_for_valley': False # Lowest point (valley) marks end of repetition
         },
    },
    'lateral_raise': {
        'trajectory_logic': {
            'metric': 'shoulder_angle', # Use Y-coordinate of hips # also possible: 'shoulder_y', 'hip_y', 'wrist_y', 'knee_angle', etc
            'joints': ['Hip', 'Shoulder', 'Wrist'], # Base joint names
            'invert_for_valley': False # Lowest point (valley) marks end of repetition
         },
    },
    'leg_extension': {
        'trajectory_logic': {
            'metric': 'ankle_y', # Use Y-coordinate of hips # also possible: 'shoulder_y', 'hip_y', 'wrist_y', 'knee_angle', etc
            'joints': ['Ankle'], # Base joint names
            'invert_for_valley': False # Lowest point (valley) marks end of repetition
         },
    },
    'leg_raises': {
        'trajectory_logic': {
            'metric': 'hip_angle', # Use Y-coordinate of hips # also possible: 'shoulder_y', 'hip_y', 'wrist_y', 'knee_angle', etc
            'joints': ['Shoulder','Hip','Ankle'], # Base joint names
            'invert_for_valley': False # Lowest point (valley) marks end of repetition
         },
    },
    'plank': {
        'trajectory_logic': {
            'metric': 'hip_y', # Use Y-coordinate of hips # also possible: 'shoulder_y', 'hip_y', 'wrist_y', 'knee_angle', etc
            'joints': ['Hip'], # Base joint names
            'invert_for_valley': False # Lowest point (valley) marks end of repetition
         },
    },
    'pull_Up': {
        'trajectory_logic': {
            'metric': 'nose_y', # Use Y-coordinate of hips # also possible: 'shoulder_y', 'hip_y', 'wrist_y', 'knee_angle', etc
            'joints': ['Nose'], # Base joint names
            'invert_for_valley': False # Lowest point (valley) marks end of repetition
         },
    },
    'push-up': {
        'trajectory_logic': {
            'metric': 'ankle_angle', # Use Y-coordinate of hips # also possible: 'shoulder_y', 'hip_y', 'wrist_y', 'knee_angle', etc
            'joints': ['Shoulder', 'Ankle', 'Wrist'], # Base joint names
            'invert_for_valley': False # Lowest point (valley) marks end of repetition
         },
    },
    'romanian_deadlift': {
        'trajectory_logic': {
            'metric': 'hip_angle', # Use Y-coordinate of hips # also possible: 'shoulder_y', 'hip_y', 'wrist_y', 'knee_angle', etc
            'joints': ['Shoulder','Hip', 'Knee'], # Base joint names
            'invert_for_valley': False # Lowest point (valley) marks end of repetition
         },
    },
    'russian_twist': {
        'trajectory_logic': {
            'metric': 'wrist_x', # Use Y-coordinate of hips # also possible: 'shoulder_y', 'hip_y', 'wrist_y', 'knee_angle', etc
            'joints': ['Wrist'], # Base joint names
            'invert_for_valley': True # Lowest point (valley) marks end of repetition
         },
    },
    'shoulder_press': {
        'trajectory_logic': {
            'metric': 'wrist_y', # Use Y-coordinate of hips # also possible: 'shoulder_y', 'hip_y', 'wrist_y', 'knee_angle', etc
            'joints': ['Wrist'], # Base joint names
            'invert_for_valley': False # Lowest point (valley) marks end of repetition
         },
    },
    't_bar_row': {
        'trajectory_logic': {
            'metric': 'elbow_angle', # Use Y-coordinate of hips # also possible: 'shoulder_y', 'hip_y', 'wrist_y', 'knee_angle', etc
            'joints': ['Shoulder', 'Elbow', 'Wrist'], # Base joint names
            'invert_for_valley': False # Lowest point (valley) marks end of repetition
         },
     },
} 

### Model Setup

This cell loads the pose estimation model (`SimpleHRNet`) and selects GPU (`cuda`) or CPU for computation based on availability.

* `DEVICE`: (`torch.device`)
    * Hardware device (CPU or CUDA-enabled GPU) that the model will run on.
* `HRNET_MODEL`: (`SimpleHRNet` object or `None`)
    * Initialized SimpleHRNet model object, if loading fails, it remains `None`.

In [14]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
HRNET_MODEL = None
try:
    model_c_value = 48
    if not Model_path.exists():
         raise FileNotFoundError(f"Model checkpoint file not found at {Model_path}")

    HRNET_MODEL = SimpleHRNet(c=model_c_value,
                              nof_joints=Expected_Joint_Count,
                              checkpoint_path=Model_path,
                              device=DEVICE,
                              multiperson=False,
                              max_batch_size=16)
    print("SimpleHRNet model loaded successfully.")
except FileNotFoundError as nofile_error:
    print(nofile_error)

device: 'cpu'
SimpleHRNet model loaded successfully.


### Subfunctions

**Purpose:**

Subfunctions for the main functions later on

* **`get_keypoint(frame_kps, kp_index)`:**
    * To retrieve the X, Y coordinates for a single joint (`kp_index`) from a single frame (`frame_kps`, reshaped to `(num_joints, 2)`).
    * Outcome: The coordinates of the joint if found and not NaN, otherwise `None`.
    * Unit = Coordinate values.
* **`calculate_angle(p1, p2, p3)`:**
    * To compute the angle at point `p2` by the vectors connecting `p1-p2` and `p3-p2`. Used for joint angles
    * Outcome: The calculated angle if all points are valid, otherwise `None`.
    * Unit = Degrees.
* **`handle_nan_values(sequence)`:**
    * Clean a pose sequence by removing frames (rows) that contain any `NaN` (can occur if one or more joints were not detected in a frame).
    * Outcome: `numpy.array` containing only the frames where all joints were detected.
    * Unit = Same as the input sequence.
* **`time_normalize_sequence(sequence, target_length)`:**
    * To standardize the length of a single repetition to a fixed number of steps (`target_length`). It resamples the sequence using linear interpolation.
    * Outcome: A `numpy.array` with shape (`target_length`, number_of_coordinates), where the movement has been stretched or compressed in time.
    * Unit = Same as the input sequence but with time axis now normalized to `target_length` steps.

In [15]:
def get_keypoint(frame_kps, kp_index):
    """Safely get keypoint coordinates (x, y) from a frame's keypoint array."""
    if frame_kps is not None and 0 <= kp_index < frame_kps.shape[0]:
        coords = frame_kps[kp_index]
        if not np.isnan(coords).any():
            return coords
    
    return None

def calculate_angle(p1, p2, p3):
    """Calculates the angle (in degrees) at p2 formed by p1-p2-p3."""
    if p1 is None or p2 is None or p3 is None: 
        return None
    
    v1, v2 = np.array(p1) - np.array(p2), np.array(p3) - np.array(p2)
    mag1, mag2 = np.linalg.norm(v1), np.linalg.norm(v2)
    if mag1 * mag2 == 0 or np.isclose(mag1 * mag2, 0): 
        return None
    
    cos_angle = np.clip(np.dot(v1, v2) / (mag1 * mag2), -1.0, 1.0)
    angle = np.degrees(np.arccos(cos_angle))    
    return angle

def handle_nan_values(sequence):
    """Handles NaN values in a pose sequence (frames, coords). Removes frames containing any NaNs."""
    if sequence is None or sequence.ndim != 2 or sequence.shape[0] == 0:
        return np.empty((0, sequence.shape[1] if sequence is not None and sequence.ndim == 2 else 0))
    
    valid_frames_mask = ~np.isnan(sequence).any(axis=1)
    return sequence[valid_frames_mask]

def time_normalize_sequence(sequence, target_length):
    """Resamples a pose sequence (frames, coords) to a target length using linear interpolation."""
    if sequence is None or sequence.shape[0] < 2:
        num_coords = sequence.shape[1] if sequence is not None and sequence.ndim == 2 else Expected_Joint_Count * 2
        return np.full((target_length, num_coords), np.nan)

    num_frames, num_coords = sequence.shape
    original_indices = np.linspace(0, num_frames - 1, num_frames)
    target_indices = np.linspace(0, num_frames - 1, target_length)
    normalized_sequence = np.zeros((target_length, num_coords))

    for j in range(num_coords):
        valid_mask = ~np.isnan(sequence[:, j])
        if np.sum(valid_mask) < 2:
            normalized_sequence[:, j] = np.nan
        else:
            try:
                normalized_sequence[:, j] = np.interp(target_indices, original_indices[valid_mask], sequence[valid_mask, j])
            except Exception: 
                normalized_sequence[:, j] = np.nan

    if np.isnan(normalized_sequence).any():
        try:
             df = pd.DataFrame(normalized_sequence)
             df.ffill(inplace=True)
             df.bfill(inplace=True)
             normalized_sequence = df.to_numpy()
        except NameError:
             normalized_sequence = np.nan_to_num(normalized_sequence, nan=0.0)

    return np.nan_to_num(normalized_sequence, nan=0.0)

### Angle Detection

This cell estimates the camera's viewing angle relative to the person in the video ('left', 'right', or 'front') using the average horizontal positions of the shoulders.

* **`detect_camera_angle(pose_sequence, joint_names, min_valid_frames=10, width_threshold_factor=0.35)`:**
    * Estimate camera angle by analyzing the average horizontal positions of the 'Left Shoulder' and 'Right Shoulder' joints over time.
    * Parameters:
        * `pose_sequence`: (`numpy.array`, Unit = Normalized coordinates) - The input pose data, expected shape (frames, num_coords).
        * `joint_names`: (`list` of `str`) - List of joint names.
        * `min_valid_frames`: (`int`, Unit = Frames) - Minimum number of frames where both shoulders must be detected for the angle detection to continue.
        * `width_threshold_factor`: (`float`, Unit = Unitless ratio) - Sensitivity. The absolute difference in average shoulder X-positions must exceed this fraction of the average detected shoulder width to be classified as a side view.
    * A string indicating the estimated angle, Unit = Category ('left', 'right', 'front', 'unknown').
    * Internal calculations:
        * Identifies frames where both left and right shoulders are reliably detected or returns 'unknown' if there are too few (`< min_valid_frames`).
        * Calculates the average X-position of each shoulder (`avg_x_left`, `avg_x_right`).
        * Calculates the average horizontal distance between the shoulders (`avg_shoulder_width`)
        * Calculates the difference (`x_difference = avg_x_right - avg_x_left`).
        * Compares the absolute `x_difference` to threshold (`avg_shoulder_width * width_threshold_factor`).
        * Return 'right' if difference is large and positive (right shoulder further right on screen) thus camera view from right
        * Return 'left' if difference is large and negative (right shoulder further left on screen) thus camera view from left
        * Otherwise return 'front' (shoulders relatively aligned horizontally) or 'unknown' if necessary joints aren't found

In [16]:
def detect_camera_angle(pose_sequence, joint_names, min_valid_frames=10, width_threshold_factor=0.35):
    """
    Analyzes pose sequence for camera angle based on shoulder positions.
    """
    if pose_sequence is None or pose_sequence.ndim != 2 or pose_sequence.shape[0] < min_valid_frames:
        return 'unknown'

    try:
        lshoulder_idx = joint_names.index('Left Shoulder')
        rshoulder_idx = joint_names.index('Right Shoulder')
        lshoulder_x_idx, rshoulder_x_idx = 2 * lshoulder_idx, 2 * rshoulder_idx

        if max(lshoulder_x_idx, rshoulder_x_idx) >= pose_sequence.shape[1]:
             return 'unknown'

        valid_mask = ~np.isnan(pose_sequence[:, lshoulder_x_idx]) & \
                     ~np.isnan(pose_sequence[:, rshoulder_x_idx])

        if np.sum(valid_mask) < min_valid_frames:
            return 'unknown'

        valid_lsx = pose_sequence[valid_mask, lshoulder_x_idx]
        valid_rsx = pose_sequence[valid_mask, rshoulder_x_idx]

        avg_x_left = np.mean(valid_lsx)
        avg_x_right = np.mean(valid_rsx)

        avg_shoulder_width = np.mean(np.abs(valid_lsx - valid_rsx))

        if avg_shoulder_width < 1e-6:
             return 'front'

        x_difference = avg_x_right - avg_x_left

        threshold = avg_shoulder_width * width_threshold_factor

        detected_angle = 'front'
        if x_difference > threshold:
            detected_angle = 'right'
        elif x_difference < -threshold:
            detected_angle = 'left'

        return detected_angle

    except ValueError:
        return 'unknown'

### Video Processing en Normalization

This cell extracts the raw pose data from a video file using the loaded HRNet model and performs normalization on the extracted raw poses.

**`process_single_video(video_path, model, Expected_Joint_Count)`:**
* To read a specified video file frame-by-frame, apply the loaded pose estimation `model` (`HRNET_MODEL`) to each frame, and extract the raw joint coordinates. It assumes a single person is the primary subject.
    * Returns a 2D NumPy array where each row represents a frame and columns contain the flattened (x, y) coordinates for all `Expected_Joint_Count` joints (shape `(num_frames, 34)` for 17 joints). 
    * Unit = Pixel coordinates (relative to the video frame dimensions).

**`normalize_pose_sequence(pose_sequence, joint_names, ref_joint1_name="Left Shoulder", ref_joint2_name="Right Shoulder")`:**
* To perform spatial normalization on the raw pose sequence obtained from `process_single_video`. This aims to make the pose data invariant to the person's location within the video frame and their scale (distance from the camera).
    * Inner logic: For each frame, it calculates a center point (midpoint between `ref_joint1` and `ref_joint2`, examples are Left/Right Shoulder) and a scale factor (distance between `ref_joint1` and `ref_joint2`). It then recalculates all joint coordinates relative to this center point and scales them by the calculated scale factor.
    * Returns a `numpy.array` of the same shape as the input `pose_sequence`, but containing the normalized coordinates.
    * Unit = Unitless normalized coordinates. The values represent positions relative to the reference joints, scaled by the distance between them.

In [17]:
def process_single_video(video_path, model, Expected_Joint_Count=Expected_Joint_Count):
    """Processes a single video, extracts joints, returns NumPy array (frames, num_coords)."""
    if not os.path.exists(video_path): 
        print(f"Video not found: {video_path}")
        return None
    
    video = cv2.VideoCapture(video_path)
    if not video.isOpened(): 
        print(f"Cannot open video: {video_path}")
        return None
    
    video_joints, frame_count = [], 0 
    expected_coords = Expected_Joint_Count * 2
    try:
        while True:
            ret, frame = video.read()
            if not ret: 
                break
            try:
                joints = model.predict(frame)
                if joints is None or joints.shape[0]==0 or joints.shape[1]!=Expected_Joint_Count: 
                    frame_coords=np.full(expected_coords,np.nan)
                else: 
                    frame_coords=joints[0,:,:2].flatten() 
                    frame_coords = frame_coords if frame_coords.shape[0]==expected_coords else np.full(expected_coords,np.nan)
            except Exception as e: 
                frame_coords = np.full(expected_coords, np.nan)
            
            video_joints.append(frame_coords)
            frame_count += 1
    finally: 
        video.release()
    return np.array(video_joints) if video_joints else None

def normalize_pose_sequence(pose_sequence, joint_names, ref_joint1_name="Left Shoulder", ref_joint2_name="Right Shoulder"):
    """Spatially normalizes a pose sequence based on reference joints."""
    if pose_sequence is None or pose_sequence.shape[0] == 0: 
        return np.array([])
    try: 
        ref_joint1_idx, ref_joint2_idx = joint_names.index(ref_joint1_name), joint_names.index(ref_joint2_name)
    except ValueError: 
        print(f"Ref joints not found in normalize")
        return np.full_like(pose_sequence, np.nan)
    
    normalized_frames = []
    num_coords = pose_sequence.shape[1]
    
    if num_coords == 0 or num_coords % 2 != 0 or num_coords != len(joint_names) * 2:
        print(f"Coordinate mismatch in normalize_pose_sequence ({num_coords} vs {len(joint_names)*2}).")
        return np.full_like(pose_sequence, np.nan)

    for frame in pose_sequence:
        try:
             if frame.shape[0] != num_coords: 
                 normalized_frames.append(np.full(num_coords, np.nan))
                 continue
             frame_joints = frame.reshape(-1, 2)
             if frame_joints.shape[0] != len(joint_names): 
                 normalized_frames.append(np.full(num_coords, np.nan))
                 continue
             ref1, ref2 = get_keypoint(frame_joints, ref_joint1_idx), get_keypoint(frame_joints, ref_joint2_idx)
            
             if ref1 is None or ref2 is None: 
                 normalized_frames.append(np.full(num_coords, np.nan))
                 continue
            
             center, scale = (np.array(ref1) + np.array(ref2)) / 2.0, np.linalg.norm(np.array(ref1) - np.array(ref2))
             if scale == 0 or np.isclose(scale, 0) or np.isnan(scale): 
                 normalized_frames.append(np.full(num_coords, np.nan))
                 continue
             normalized_frames.append(((frame_joints - center) / scale).flatten())
        except Exception as e:
            normalized_frames.append(np.full(num_coords, np.nan))
            continue
    
    return np.array(normalized_frames)

### Trajectory Calculation

This cell is to take a pose sequence andmcalculates a 1-dimensional time series that represents the key repetitive movement of that exercise.

**`calculate_trajectory(pose_sequence, joint_names, trajectory_config)`:**
* Calculate the 1D trajectory signal according to the rules specified in the `trajectory_config` dictionary. It handles different calculation methods ('metrics') like joint angles or coordinate positions
    *  Parameters:
        * `pose_sequence`: (`numpy.array`, Unit = Normalized coordinates) - The cleaned, spatially normalized pose sequence (frames, coords).
        * `joint_names`: (`list` of `str`) - Ordered list of joint names.
        * `trajectory_config`: (`dict`) - The specific `trajectory_logic` dictionary for the current exercise, extracted from `EXERCISE_CONFIG`.
    * Outcome: Returns a tuple containing:
        * `trajectory`: (`numpy.array` or `None`) - A 1D NumPy array `(num_frames,)` with the calculated metric over time. Returns `None` if fails.
        * `invert_for_valley`: (`bool`) - Indicating if the peak finding logic should invert this trajectory to find valleys.
    * Unit = degrees for `_angle` metrics, normalized units for `_y` or `_x` metrics).


In [18]:
def calculate_trajectory(pose_sequence, joint_names, trajectory_config):
    """Calculates the 1D trajectory signal based on configuration."""
    if pose_sequence is None or pose_sequence.ndim!= 2 or pose_sequence.shape[0] < 2: 
        return None, False

    metric = trajectory_config.get('metric', 'unknown')
    base_joint_names = trajectory_config.get('joints', [])
    invert_for_valley = trajectory_config.get('invert_for_valley', False)
    trajectory = None

    try:
        if not base_joint_names: 
            raise ValueError("No 'joints' specified in trajectory_config")

        if metric.endswith('_angle'):
            if len(base_joint_names) != 3: 
                raise ValueError(f"{metric} needs 3 joints in config")
            
            j1_base, j2_base, j3_base = base_joint_names[0], base_joint_names[1], base_joint_names[2]
            angles_left, angles_right = [], []
            try: 
                l_j1_idx = joint_names.index(f'Left {j1_base}')
                l_j2_idx = joint_names.index(f'Left {j2_base}')
                l_j3_idx = joint_names.index(f'Left {j3_base}')
                r_j1_idx = joint_names.index(f'Right {j1_base}')
                r_j2_idx = joint_names.index(f'Right {j2_base}')
                r_j3_idx = joint_names.index(f'Right {j3_base}')
                indices_valid = True
            except (ValueError, IndexError): 
                indices_valid = False 

            if indices_valid:
                for frame_coords in pose_sequence:
                    f_j = frame_coords.reshape(-1, 2)
                    p1l,p2l,p3l = get_keypoint(f_j,l_j1_idx), get_keypoint(f_j,l_j2_idx), get_keypoint(f_j,l_j3_idx)
                    p1r,p2r,p3r = get_keypoint(f_j,r_j1_idx), get_keypoint(f_j,r_j2_idx), get_keypoint(f_j,r_j3_idx)
                    angles_left.append(calculate_angle(p1l, p2l, p3l) if all(p is not None for p in [p1l, p2l, p3l]) else np.nan)
                    angles_right.append(calculate_angle(p1r, p2r, p3r) if all(p is not None for p in [p1r, p2r, p3r]) else np.nan)
                traj_l, traj_r = np.array(angles_left), np.array(angles_right)
                trajectory = np.nanmean([traj_l, traj_r], axis=0) 
            else: 
                 j1_idx, j2_idx, j3_idx = joint_names.index(j1_base), joint_names.index(j2_base), joint_names.index(j3_base)
                 angles = []
                 for frame_coords in pose_sequence:
                      f_j = frame_coords.reshape(-1, 2)
                      p1, p2, p3 = get_keypoint(f_j, j1_idx), get_keypoint(f_j, j2_idx), get_keypoint(f_j, j3_idx)
                      angles.append(calculate_angle(p1, p2, p3) if all(p is not None for p in [p1, p2, p3]) else np.nan)
                 trajectory = np.array(angles)

        elif metric.endswith('_y') or metric.endswith('_x'):
            coord_idx = 1 if metric.endswith('_y') else 0 
            base_joint = base_joint_names[0]
            try: 
                 l_joint_idx = joint_names.index(f'Left {base_joint}')
                 r_joint_idx = joint_names.index(f'Right {base_joint}')
                 if max(2*l_joint_idx+coord_idx, 2*r_joint_idx+coord_idx) >= pose_sequence.shape[1]: 
                     raise ValueError("Joint index out of bounds")
                 l_coord = pose_sequence[:, 2 * l_joint_idx + coord_idx]
                 r_coord = pose_sequence[:, 2 * r_joint_idx + coord_idx]
                 trajectory = np.nanmean([l_coord, r_coord], axis=0)
            except (ValueError, IndexError):
                 joint_idx = joint_names.index(base_joint)
                 if 2*joint_idx+coord_idx >= pose_sequence.shape[1]: 
                     raise ValueError("Joint index out of bounds")
                 trajectory = pose_sequence[:, 2 * joint_idx + coord_idx]
        else:
             raise ValueError(f"Unknown metric type given in config: '{metric}'")

        if trajectory is None or trajectory.shape[0] == 0 or np.all(np.isnan(trajectory)): 
            return None, False
        nan_mask = np.isnan(trajectory)
        if np.any(nan_mask):
            indices = np.arange(len(trajectory)) 
            valid_indices = np.flatnonzero(~nan_mask)
            if len(valid_indices) < 2: 
                return None, False
            trajectory[nan_mask] = np.interp(indices[nan_mask], valid_indices, trajectory[valid_indices])
        if np.any(np.isnan(trajectory)):
             trajectory = np.nan_to_num(trajectory, nan=np.nanmean(trajectory))
             if np.any(np.isnan(trajectory)): trajectory = np.nan_to_num(trajectory, nan=0.0)
        return trajectory, invert_for_valley

    except (ValueError, IndexError) as e: 
        print(f"Error finding/using joints for metric '{metric}': {e}")
        return None, False
    except Exception as e: 
        print(f"Error calculating trajectory with metric '{metric}': {e}")
        return None, False

### Extract trajectory

This cell takes a numerical trajectory (movement over time) and identify the locations (frame indices) of significant peaks within that signal. By using the `invert_trajectory`, it can also find significant valleys. This peak/valley detection is used for identifying individual repetition boundaries in the functions `segment_repetitions` and `count_repetitions`.

1.  **`extract_trajectory(trajectory, smoothing_window, prominence, distance, invert_trajectory=False)`:**
    * To smooth an input trajectory signal and find the indices of peaks that meet specific prominence and distance criteria.
    *   Parameters:
        * `trajectory`: (`numpy.array`, Unit = Varies - degrees, normalized units) - The 1D input signal calculated by `calculate_trajectory`.
        * `smoothing_window`: (`int`, Unit = Frames) - The number of frames to use for the moving average filter applied before peak detection.
        * `prominence`: (`float`, Unit = Unitless ratio) - Relative prominence factor. A peak must stand out from its surrounding troughs by at least this fraction of the smoothed signal's overall range.
        * `distance`: (`int`, Unit = Frames) - The minimum number of frames required between consecutive detected peaks.
        * `invert_trajectory`: (`bool`) - If `True`, the trajectory is inverted (`-trajectory`) before smoothing and peak finding. This effectively finds significant valleys (local minima) in the original signal.
    * A 1D NumPy array containing the frame indices where valid peaks were detected in the unsmoothed trajectory's timeline, or `None` if the input is invalid, smoothing/peak finding fails, or no peaks are found.
    * Unit = Frame index (unitless count).

In [19]:
def extract_trajectory(trajectory, smoothing_window, prominence, distance, invert_trajectory=False):
    """detect a trajectory and find peaks."""
    if trajectory is None or trajectory.ndim != 1 or trajectory.shape[0] < max(distance * 2, smoothing_window, 1): 
        return None

    processed_trajectory = trajectory.copy()
    if invert_trajectory: processed_trajectory = -processed_trajectory
    if len(processed_trajectory) < smoothing_window: 
        return None

    smoothed_trajectory = np.convolve(processed_trajectory, np.ones(smoothing_window)/smoothing_window, mode='valid')
    if smoothed_trajectory.shape[0] < max(distance, 1): 
        return None

    data_range = np.ptp(smoothed_trajectory)
    required_prominence = max(data_range * prominence, 1e-6) if data_range > 1e-9 else 1e-6

    try:
        smoothed_indices_offset = (len(processed_trajectory) - len(smoothed_trajectory)) // 2
        peak_indices_smoothed, _ = find_peaks(smoothed_trajectory, prominence=required_prominence, distance=distance)
        peak_indices_original = peak_indices_smoothed + smoothed_indices_offset
        peak_indices_original = peak_indices_original[(peak_indices_original >= 0) & (peak_indices_original < len(processed_trajectory))]
        return peak_indices_original
    except Exception as e: 
        print(f"Peak finding failed: {e}") 
        return None

### Segmentation and Counting

This cell is analyzing the movement trajectory to identify repetitions.

1.  **`segment_repetitions(pose_sequence, exercise_label, joint_names, exercise_config, smoothing_window=Smoothing_window_size, peak_prominence=Default_Peak_Prominence, peak_distance=Default_Peak_Distance)`:**
    * To divide a given `pose_sequence` (assumed to contain multiple repetitions of the specified `exercise_label`) into a list of smaller sequences, where each smaller sequence corresponds to a single detected repetition.
    *   Parameters:
        * `pose_sequence`: (`numpy.array`, Unit = Normalized coordinates) - The input sequence to segment.
        * `exercise_label`: (`str`) - The name of the exercise, used to look up the correct processing logic in `exercise_config`.
        * `joint_names`: (`list` of `str`) - Ordered list of joint names.
        * `exercise_config`: (`dict`) - The main configuration dictionary containing trajectory logic for all exercises.
        * `smoothing_window`, `peak_prominence`, `peak_distance`: (`int`/`float`, Unit = Frames/Unitless ratio) - Parameters controlling the peak detection process
    * A list, where each element is a `numpy.array` representing the pose data for a single detected repetition segment (shape `(segment_frames, num_coords)`). Returns an empty list if no valid repetitions are found or if the configuration is missing.
    * Unit = Each array contains normalized coordinates.

2.  **`count_repetitions(pose_sequence, exercise_label, joint_names, exercise_config, prominence=Default_Peak_Prominence, distance=Default_Peak_Distance, smoothing_window=Smoothing_window_size)`:**
    * To estimate the total number of repetitions within the input `pose_sequence` for the given `exercise_label` using `prominence` and `distance` parameters (values found during baseline loading).
    *   Parameters:
        * `pose_sequence`: (`numpy.array`, Unit = Normalized coordinates) - The input sequence to analyze.
        * `exercise_label`: (`str`) - The name of the exercise.
        * `joint_names`: (`list` of `str`) - Ordered list of joint names.
        * `exercise_config`: (`dict`) - The main configuration dictionary.
        * `prominence`, `distance`, `smoothing_window`: (`float`/`int`, Unit = Unitless ratio/Frames) - Peak detection parameters.
    * The estimated number of repetitions detected in the sequence. Returns 0 if the config is missing or no peaks are found.
    * Unit = Count (unitless).

In [20]:
def segment_repetitions(pose_sequence, exercise_label, joint_names, exercise_config, smoothing_window=Smoothing_window_size,
                        peak_prominence=Default_Peak_Prominence, peak_distance=Default_Peak_Distance):
    """Segments a pose sequence into individual repetitions using EXERCISE_CONFIG."""
    
    current_exercise_cfg_dict = exercise_config.get(exercise_label, {})
    trajectory_logic_cfg = current_exercise_cfg_dict.get('trajectory_logic')
    if not trajectory_logic_cfg:
        return []

    trajectory, invert = calculate_trajectory(pose_sequence, joint_names, trajectory_logic_cfg)
    peak_indices = extract_trajectory(trajectory, smoothing_window, peak_prominence, peak_distance, invert_trajectory=invert)

    if peak_indices is None or len(peak_indices) < 2: 
        return []
    repetitions = [pose_sequence[peak_indices[i]:peak_indices[i+1], :] for i in range(len(peak_indices) - 1)]
    return [rep for rep in repetitions if rep.shape[0] > 1]

def count_repetitions(pose_sequence, exercise_label, joint_names, exercise_config, prominence=Default_Peak_Prominence, distance=Default_Peak_Distance,
                      smoothing_window=Smoothing_window_size):
    """Counts repetitions using EXERCISE_CONFIG and specified parameters."""
    
    current_exercise_cfg_dict = exercise_config.get(exercise_label)
    if not current_exercise_cfg_dict: 
        return 0
    trajectory_logic_cfg = current_exercise_cfg_dict.get('trajectory_logic')
    if not trajectory_logic_cfg: 
        return 0

    trajectory, invert = calculate_trajectory(pose_sequence, joint_names, trajectory_logic_cfg)
    peak_indices = extract_trajectory(trajectory, smoothing_window, prominence, distance, invert_trajectory=invert)

    return len(peak_indices) if peak_indices is not None else 0

### Parameter Tuning Function

This cell finds the optimal `peak_prominence` and `peak_distance` values for the repetition counting. It does this by trying out different parameter combinations on the baseline videos for per exercise and selecting the parameter pair that minimizes the total counting error.

1.  **`tune_counting_parameters(exercise_label, labeled_video_data, joint_names, exercise_config, prominence_range, distance_range)`:**
    * To perform a grid search over `prominence_range` and `distance_range` to find the combination of `peak_prominence` and `peak_distance` that most accurately counts repetitions across the provided `labeled_video_data` for the given `exercise_label`.
    *  Parameters:
        * `exercise_label`: (`str`) - The name of the exercise being tuned.
        * `labeled_video_data`: (`list` of `tuple`) - Data for tuning. Each element is `(video_index, (cleaned_pose_sequence, true_reps))`. The `cleaned_pose_sequence` is the spatially normalized sequence *before* time normalization. `true_reps` is the ground truth count parsed from the filename.
        * `joint_names`: (`list` of `str`) - Ordered list of joint names.
        * `exercise_config`: (`dict`) - The main configuration dictionary.
        * `prominence_range`: (`list` of `float`) - List of `peak_prominence` values to test.
        * `distance_range`: (`list` of `int`) - List of `peak_distance` values to test.
    * Returns a tuple containing the best `(prominence, distance)` parameters found for this exercise and prints summary of search process
    * Unit = Prominence is a unitless ratio, Distance is in frames.

In [21]:
def tune_counting_parameters(exercise_label, labeled_video_data, joint_names, exercise_config, prominence_range, distance_range):
    """Performs grid search using the refactored peak finding and config."""
    print(f" Tuning repetition counting parameters for: {exercise_label}...")
    
    best_params = (Default_Peak_Prominence, Default_Peak_Distance)
    
    min_total_error = float('inf')
    
    if not labeled_video_data: 
        print("No labeled data provided for tuning.")
        return best_params

    current_exercise_cfg_dict = exercise_config.get(exercise_label)
    
    if not current_exercise_cfg_dict: 
        print(f"No config for {exercise_label} in tuning")
        return best_params
    
    trajectory_logic_cfg = current_exercise_cfg_dict.get('trajectory_logic')
    
    if not trajectory_logic_cfg: 
        print(f"No trajectory_logic in config for {exercise_label}")
        return best_params

    num_combinations = len(prominence_range) * len(distance_range)
    
    default_error = 0
    
    precalculated_trajectories = {}

    for i, (vid_idx, (sequence, true_reps)) in enumerate(labeled_video_data): 
        if sequence is None or sequence.ndim != 2 or sequence.shape[0] < 2: 
            continue
        trajectory, invert = calculate_trajectory(sequence, joint_names, trajectory_logic_cfg)
        
        if trajectory is not None: 
            precalculated_trajectories[vid_idx] = (trajectory, invert, true_reps)

    if not precalculated_trajectories: 
        print("No valid trajectories calculated for tuning.") 
        return best_params

    for vid_idx, (traj, inv, true_r) in precalculated_trajectories.items():
         peak_indices_def = extract_trajectory(traj, Smoothing_window_size, Default_Peak_Prominence, Default_Peak_Distance, invert_trajectory=inv)
         calc_reps_def = len(peak_indices_def) if peak_indices_def is not None else 0
         default_error += abs(calc_reps_def - true_r)

    # Grid search
    for p_idx, p in enumerate(prominence_range):
        for d_idx, d in enumerate(distance_range):
            current_total_error = 0
            for vid_idx, (traj, inv, true_r) in precalculated_trajectories.items():
                peak_indices = extract_trajectory(traj, Smoothing_window_size, p, d, invert_trajectory=inv)
                calculated_reps = len(peak_indices) if peak_indices is not None else 0
                current_total_error += abs(calculated_reps - true_r)

            if current_total_error < min_total_error:
                min_total_error = current_total_error
                best_params = (p, d)
            if min_total_error == 0: 
                break
        if min_total_error == 0: 
            break

    print(f"Tuning complete. Best Params: P={best_params[0]:.3f}, D={best_params[1]}, Min Error={min_total_error}. (Error with default parameters = {default_error})")

    if precalculated_trajectories:
        print(f"Detected reps using Best params (P={best_params[0]:.3f}, D={best_params[1]}):")
        total_calculated, total_true = 0, 0

        for vid_idx, (traj, inv, true_r) in sorted(precalculated_trajectories.items()):
            peak_indices = extract_trajectory(traj, Smoothing_window_size, best_params[0], best_params[1], invert_trajectory=inv)
            calculated_reps = len(peak_indices) if peak_indices is not None else 0
            print(f"Video Index {vid_idx}: Detected={calculated_reps}, True={true_r}")   ###### nog naam van videofile nog toevoegen/vervangen
            total_calculated += calculated_reps
            total_true += true_r
        print(f"Total Reps: Detected={total_calculated}, True={total_true}, Overall Error={abs(total_calculated - total_true)}")
        
    return best_params

### Loading baseline videos

This cell processing all baseline video files. It iterates through each exercise folder provided in `Video_baseline_path`.
* Inner logic:
    * Parses Information: Extracts the ground truth repetition count (`XX`) and camera angle (`'left'`, `'right'`, or `'front'`) from each baseline video's filename (using the format `exercise_label_XXreps_angle_something.mp4`).
    * Processes Videos: Extracts and normalizes pose sequences for each video.
    * Segment Repetitions: Breaks down each video's cleaned sequence into individual repetition segments
    * Groups by Angle: Groups the time-normalized repetition segments + labeled data (sequence + true reps) based on the `parsed_angle` for each exercise.
    * Tunes Parameters per Angle: For each angle category ('left', 'right', 'front', 'unknown') finds the optimal peak detection settings.
    * Averages Baselines per Angle: For each angle category with data, it calculates and stores the average time-normalized repetition sequence.
    * Derives Thresholds per Angle: For each angle category, it calculates a suggested DTW threshold based on the variation between individual reps
    * Calculates Overall Threshold: After processing all angles, it averages all valid reps per exercise to create an 'overall' baseline threshold.

1.  **`load_baseline_data(Video_baseline_path, hrnet_model, joint_names, Target_rep_length, exercise_config, prominence_range, distance_range, threshold_std_multiplier=Threshold_STD_Multiplier, min_reps_for_threshold=Min_REPS_to_calc_threshold)`:**
    * Load baseline videos, organize data by exercise and parsed filename angle, tune counting parameters per angle, create averaged baselines per angle and overall
    * Outcome: Returns a tuple containing three nested dictionaries:
        1.  `averaged_baseline_data`: (`dict`) Structure: `{exercise_label: {angle_key: numpy.array}}`. Contains the averaged time-normalized pose sequence for each exercise under keys for detected angles ('left', 'right', 'front', 'unknown') and 'overall'.
        2.  `tuned_counting_params`: (`dict`) Structure: `{exercise_label: {angle_key: (prominence, distance)}}`. Contains the best `(prominence, distance)` tuple found for counting reps for each exercise/angle combination.
        3.  `exercise_specific_thresholds`: (`dict`) Structure: `{exercise_label: {angle_key: float or None}}`. Contains the derived DTW threshold calculated for each exercise/angle combination. Value is `None` if too few reps.

In [22]:
def load_baseline_data(Video_baseline_path, hrnet_model, joint_names, Target_rep_length, exercise_config, prominence_range, distance_range,
                       threshold_std_multiplier=Threshold_STD_Multiplier, min_reps_for_threshold=Min_REPS_to_calc_threshold):
    """
    Loads baselines, parses angle from filename, tunes params per angle, averages per angle and overall and calculates thresholds.
    Returns dicts: averaged_baselines, tuned_counting_params, exercise_specific_thresholds
    """
    averaged_baseline_data = {}
    tuned_counting_params = {}
    exercise_specific_thresholds = {}
    start_time_total = time.time()

    print(f"Loading Baselines (Config-Driven, Angle from Filename)... from: {Video_baseline_path}")
    if not os.path.isdir(Video_baseline_path):
        print(f"Baseline directory not found: {Video_baseline_path}")
        return averaged_baseline_data, tuned_counting_params, exercise_specific_thresholds

    for exercise_label_raw in sorted(os.listdir(Video_baseline_path)):
        exercise_folder_path = os.path.join(Video_baseline_path, exercise_label_raw)
        if not os.path.isdir(exercise_folder_path): 
            continue
        exercise_label = exercise_label_raw.lower().replace(" ", "_")
        print(f"\nProcessing baseline exercise: {exercise_label} (from folder: {exercise_label_raw})")

        current_exercise_config_dict = exercise_config.get(exercise_label)
        if not current_exercise_config_dict:
            print(f" No config found for '{exercise_label}'. Skipping.") 
            continue

        all_reps_by_angle = {'left': [], 'right': [], 'front': [], 'unknown': []}
        labeled_data_by_angle = {'left': [], 'right': [], 'front': [], 'unknown': []} # Stores (vid_idx, (sequence, true_reps))
        averaged_baseline_data[exercise_label] = {}
        tuned_counting_params[exercise_label] = {}
        exercise_specific_thresholds[exercise_label] = {}

        video_files = sorted([f for f in os.listdir(exercise_folder_path) if f.lower().endswith(('.mp4', '.avi', '.mov'))])
        print(f"Found {len(video_files)} video file(s).")

        for video_idx, video_file in enumerate(video_files):
            video_path = os.path.join(exercise_folder_path, video_file)
            true_reps, parsed_angle = None, 'unknown'
            rep_match = re.search(r'_(\d+)reps', video_file, re.IGNORECASE)
            if rep_match: true_reps = int(rep_match.group(1))
            angle_match = re.search(r'_(left|right|front)', video_file, re.IGNORECASE)
            if angle_match: parsed_angle = angle_match.group(1).lower()

            raw_seq = process_single_video(video_path, hrnet_model)
            
            if raw_seq is None or raw_seq.shape[0] == 0: 
                continue
            
            spatially_norm_seq = normalize_pose_sequence(raw_seq, joint_names)
            
            if spatially_norm_seq is None or np.all(np.isnan(spatially_norm_seq)): 
                continue
            cleaned_seq = handle_nan_values(spatially_norm_seq)
            if cleaned_seq.shape[0] < 2: 
                continue

            if true_reps is not None:
                 labeled_data_by_angle[parsed_angle].append((video_idx, (cleaned_seq, true_reps)))

            repetitions = segment_repetitions(cleaned_seq, exercise_label, joint_names, exercise_config)
            if not repetitions: 
                continue

            for rep_segment in repetitions:
                if rep_segment is not None and rep_segment.shape[0] >= 2:
                    time_norm_rep = time_normalize_sequence(rep_segment, Target_rep_length)
                    expected_coords = len(joint_names) * 2
                    if not np.isnan(time_norm_rep).all() and time_norm_rep.shape == (Target_rep_length, expected_coords):
                        all_reps_by_angle[parsed_angle].append(time_norm_rep)

        all_reps_across_angles = []
        for angle in ['left', 'right', 'front', 'unknown']:
            labeled_data_for_this_angle = labeled_data_by_angle[angle]
            normalized_reps_for_this_angle = all_reps_by_angle[angle]
            
            if not normalized_reps_for_this_angle: 
                continue
                
            all_reps_across_angles.extend(normalized_reps_for_this_angle)

            if labeled_data_for_this_angle:
                 tuned_p, tuned_d = tune_counting_parameters(exercise_label, labeled_data_for_this_angle,
                                                            joint_names, exercise_config, prominence_range, distance_range)
                 tuned_counting_params[exercise_label][angle] = (tuned_p, tuned_d)
            else: 
                tuned_counting_params[exercise_label][angle] = (Default_Peak_Prominence, Default_Peak_Distance)

            reps_stack = np.stack(normalized_reps_for_this_angle, axis=0)
            averaged_sequence_angle = np.nanmean(reps_stack, axis=0)
            averaged_baseline_data[exercise_label][angle] = np.nan_to_num(averaged_sequence_angle, nan=0.0)

            derived_threshold_angle = None
            if len(normalized_reps_for_this_angle) >= min_reps_for_threshold:
                intra_distances_angle = []
                safe_avg_angle = np.nan_to_num(averaged_sequence_angle, nan=0.0)
                for norm_rep in normalized_reps_for_this_angle:
                     safe_rep = np.nan_to_num(norm_rep, nan=0.0)
                     if safe_rep.shape == safe_avg_angle.shape:
                         try: 
                             dist, _ = fastdtw(safe_rep, safe_avg_angle, dist=euclidean) 
                             intra_distances_angle.append(dist)
                         except Exception: 
                             pass
                if len(intra_distances_angle) >= min_reps_for_threshold:
                     mean_dist, std_dist = np.mean(intra_distances_angle), np.std(intra_distances_angle)
                     derived_threshold_angle = mean_dist + threshold_std_multiplier * std_dist
            exercise_specific_thresholds[exercise_label][angle] = derived_threshold_angle

        if len(all_reps_across_angles) >= min_reps_for_threshold:
             overall_reps_stack = np.stack(all_reps_across_angles, axis=0)
             overall_averaged_sequence = np.nanmean(overall_reps_stack, axis=0)
             averaged_baseline_data[exercise_label]['overall'] = np.nan_to_num(overall_averaged_sequence, nan=0.0)
             overall_intra_distances = []
             safe_overall_avg = np.nan_to_num(overall_averaged_sequence, nan=0.0)
             for norm_rep in all_reps_across_angles:
                  safe_rep = np.nan_to_num(norm_rep, nan=0.0)
                  if safe_rep.shape == safe_overall_avg.shape:
                       try: 
                           dist, _ = fastdtw(safe_rep, safe_overall_avg, dist=euclidean) 
                           overall_intra_distances.append(dist)
                       except Exception: 
                           pass
             if len(overall_intra_distances) >= min_reps_for_threshold:
                  mean_dist, std_dist = np.mean(overall_intra_distances), np.std(overall_intra_distances)
                  derived_threshold_overall = mean_dist + threshold_std_multiplier * std_dist
                  exercise_specific_thresholds[exercise_label]['overall'] = derived_threshold_overall
             else: 
                 exercise_specific_thresholds[exercise_label]['overall'] = None
        else:
             averaged_baseline_data[exercise_label]['overall'] = None
             exercise_specific_thresholds[exercise_label]['overall'] = None

    end_time_total = time.time()
    print(f"\nBaseline processing complete in {time.time() - start_time_total:.2f}s.")
    return averaged_baseline_data, tuned_counting_params, exercise_specific_thresholds

### Recognition Function

This cell takes a new video file and compares it against the baseline data to classify the exercise performed in each detected repetition.

1.  **`recognize_exercise(new_video_path, averaged_baseline_data, hrnet_model, joint_names, exercise_config, Target_rep_length, tuned_counting_params, exercise_specific_thresholds, hint_angle=None, fallback_threshold=DTW_Distance_threshold)`:**
    * To process a new video, segment it into repetitions (using parameters potentially tuned for an initial best guess exercise and the hinted angle), compare each time-normalized repetition segment against angle-specific (or overall) averaged baselines using DTW, apply exercise- and angle-specific (or overall/fallback) thresholds, and return the classification result for each repetition.
    * Parameters:
        * `new_video_path`: (`str` or `pathlib.Path`) - Path to the video file to be analyzed.
        * `averaged_baseline_data`: (`dict`) - Nested dictionary `{exercise: {angle: avg_sequence}}` returned by `load_baseline_data`.
        * `hrnet_model`: (`SimpleHRNet` object) - The loaded pose estimation model.
        * `joint_names`: (`list` of `str`) - Ordered list of joint names.
        * `exercise_config`: (`dict`) - The main configuration dictionary defining exercise-specific logic.
        * `Target_rep_length`: (`int`, Unit = Frames) - Target length for time normalization.
        * `tuned_counting_params`: (`dict`) - Nested dictionary `{exercise: {angle: (prominence, distance)}}` returned by `load_baseline_data`.
        * `exercise_specific_thresholds`: (`dict`) - Nested dictionary `{exercise: {angle: threshold}}` returned by `load_baseline_data`.
        * `hint_angle`: (`str` or `None`, Unit = Category label) - Optional hint ('left', 'right', 'front'). If provided, comparison uses this angle's baselines/thresholds. If `None`, uses 'overall' baselines/thresholds. Default is `None`.
        * `fallback_threshold`: (`float`, Unit = Unitless Distance) - The global DTW threshold used if no specific derived threshold is available for the exercise/angle combination being compared.
    * Outcome: Returns a tuple containing:
        1.  `repetition_results`: (`list` of `tuple`) - A list with tuples representing a detected repetition: `(rep_index, matched_label, min_distance)`.
            * `rep_index`: (`int`) index.
            * `matched_label`: (`str`) Contains the name of the matched exercise if `min_distance` was below the relevant threshold, otherwise "No Match / Inconclusive (...)"
            * `min_distance`: (`float`) The lowest DTW distance found when comparing this repetition against the baselines.
        2.  `num_detected_reps`: (`int`) - The total number of segments detected in the video

In [23]:
def recognize_exercise(new_video_path, averaged_baseline_data, hrnet_model, joint_names, exercise_config,
                       Target_rep_length, tuned_counting_params, exercise_specific_thresholds,
                       hint_angle=None, fallback_threshold=DTW_Distance_threshold):
    """Recognizes exercise"""
    print(f"\n Recognizing Exercise (Angle Hint='{hint_angle}') for: {os.path.basename(new_video_path)} ---")
    start_time = time.time()
    if tuned_counting_params is None: 
        tuned_counting_params = {}
    if exercise_specific_thresholds is None: 
        exercise_specific_thresholds = {}
    default_p, default_d = Default_Peak_Prominence, Default_Peak_Distance

    new_raw_seq = process_single_video(new_video_path, hrnet_model)
    if new_raw_seq is None or new_raw_seq.shape[0] == 0: 
        print("Failed processing video.")
        return [], 0
    new_spatially_norm_seq = normalize_pose_sequence(new_raw_seq, joint_names)
    if new_spatially_norm_seq is None or np.all(np.isnan(new_spatially_norm_seq)): 
        print("Error")
        return [], 0
    new_sequence_clean = handle_nan_values(new_spatially_norm_seq)
    if new_sequence_clean.shape[0] < 2: 
        print("Error") 
        return [], 0

    new_sequence_time_norm = time_normalize_sequence(new_sequence_clean, Target_rep_length)
    if np.isnan(new_sequence_time_norm).all(): 
        print("Error: Time normalization failed for whole sequence.")
        return [], 0
    new_sequence_time_norm = np.nan_to_num(new_sequence_time_norm, nan=0.0)

    initial_min_distance, initial_best_label = float('inf'), None
    segment_p, segment_d = default_p, default_d
    segment_exercise_config_dict = None

    if not averaged_baseline_data: 
        print("Error: No baseline data.")
        return [], 0
        
    for ex_label, angle_dict in averaged_baseline_data.items():        
        avg_base_seq = angle_dict.get('overall')
        
        if avg_base_seq is None: 
            continue
        
        if avg_base_seq.shape[0] != Target_rep_length or avg_base_seq.ndim != 2 or new_sequence_time_norm.shape[1] != avg_base_seq.shape[1]: 
            continue
        
        safe_base = np.nan_to_num(avg_base_seq, nan=0.0)
        
        try: 
            dist, _ = fastdtw(new_sequence_time_norm, safe_base, dist=euclidean)
        except Exception: 
            continue
        
        if dist < initial_min_distance: 
            initial_min_distance, initial_best_label = dist, ex_label

    angle_for_tuning_lookup = hint_angle if hint_angle in ['left', 'right', 'front'] else 'front'

    if initial_best_label:
        segment_exercise_config_dict = exercise_config.get(initial_best_label)
        if segment_exercise_config_dict:
             tuned_p, tuned_d = tuned_counting_params.get(initial_best_label, {}).get(angle_for_tuning_lookup, (default_p, default_d))
             segment_p, segment_d = tuned_p, tuned_d
        else:
             print(f"No configuration for initial guess '{initial_best_label}'. Using default parameters.")
             initial_best_label = None
    else:
        print(f"Initial guess failed. Using default")

    segment_label_for_logic = initial_best_label if initial_best_label else "unknown"
    test_repetitions = segment_repetitions(new_sequence_clean, segment_label_for_logic, joint_names, exercise_config,
                                           peak_prominence=segment_p, peak_distance=segment_d)
    num_detected_reps = len(test_repetitions)
    print(f"Detected {num_detected_reps} potential repetitions in test video.")
    
    if num_detected_reps == 0: 
        return [], 0

    repetition_results = []
    comparison_angle_key = hint_angle if hint_angle in ['left', 'right', 'front'] else 'overall'
    print(f"Comparing each detected repetition against '{comparison_angle_key}' baselines...")

    for i, rep_segment in enumerate(test_repetitions):
        if rep_segment is None or rep_segment.shape[0] < 2: 
            repetition_results.append( (i+1, "Error: Invalid Segment", float('inf')) )
            continue
        
        time_norm_rep_segment = time_normalize_sequence(rep_segment, Target_rep_length)
        if np.isnan(time_norm_rep_segment).all(): 
            repetition_results.append( (i+1, "Error: Time Norm Failed", float('inf')) )
            continue
        time_norm_rep_segment = np.nan_to_num(time_norm_rep_segment, nan=0.0)

        min_rep_distance, best_rep_label_for_this_rep = float('inf'), None
        for exercise_label, angle_dict in averaged_baseline_data.items():
            baseline_to_compare = angle_dict.get(comparison_angle_key)
            if baseline_to_compare is None or time_norm_rep_segment.shape != baseline_to_compare.shape: 
                continue

            safe_baseline = np.nan_to_num(baseline_to_compare, nan=0.0)
            try: 
                distance, _ = fastdtw(time_norm_rep_segment, safe_baseline, dist=euclidean)
            except Exception as e: 
                print(f"DTW error Rep {i+1} vs {exercise_label}/{comparison_angle_key}: {e}")
                continue
            
            if distance < min_rep_distance: 
                min_rep_distance, best_rep_label_for_this_rep = distance, exercise_label

        matched_label_for_rep = "No Match / Inconclusive"
        threshold_to_use = fallback_threshold
        if best_rep_label_for_this_rep:
            threshold_specific = exercise_specific_thresholds.get(best_rep_label_for_this_rep, {}).get(comparison_angle_key)
            if threshold_specific is None and comparison_angle_key != 'overall': 
                 threshold_specific = exercise_specific_thresholds.get(best_rep_label_for_this_rep, {}).get('overall')

            if threshold_specific is not None: 
                 threshold_to_use = threshold_specific
            if min_rep_distance < threshold_to_use:
                 matched_label_for_rep = best_rep_label_for_this_rep
            else: 
                 matched_label_for_rep = f"No Match (Closest: {best_rep_label_for_this_rep}, Dist: {min_rep_distance:.2f} >= {threshold_to_use:.2f})"

        repetition_results.append( (i+1, matched_label_for_rep, min_rep_distance) )

    return repetition_results, num_detected_reps

### Load baseline data

This cell executes the baseline creation and processing pipeline

1.  `averaged_baselines`: (`dict`)
    * Contains the final averaged, time-normalized pose sequence for each detected angle ('left', 'right', 'front', 'unknown') and the 'overall' average for each exercise.
    * `{exercise_label: {angle_key: numpy.array}}` (`{'squat': {'front': array([...]), 'overall': array([...])}}`)
    * Unit = Arrays contain unitless normalized coordinates.
2.  `tuned_params`: (`dict`)
    * Stores the optimal 'peak_prominence' and 'peak_distance' tuple for each exercise and angle category based on the labeled baseline videos.
    * Structure: `{exercise_label: {angle_key: (prominence, distance)}}` (`{'squat': {'front': (0.06, 15), 'overall': (0.05, 10)}}`)
    * Unit = Prominence (float) is unitless ratio, Distance (int) is in frames.
4.  `derived_thresholds`: (`dict`)
    * Stores the DTW recognition threshold calculated for each exercise and angle category (including 'overall') based on the variance within repetitions (Mean + N * stddev of intra-exercise distances). Contains `None` if a threshold couldn't be derived (too few repetitions).
    * Structure: `{exercise_label: {angle_key: threshold_value or None}}` (`{'squat': {'front': 450.7, 'overall': 510.2}}`)
    * Unit = Float, unitless distance (same scale as DTW results).

In [24]:
def learned_data_to_excel(config_data, tuned_params_data, threshold_data, filename, averaged_baseline_data,output_dir="."):
    """
    Exports the exercise configuration, tuned parameters and derived thresholds to separate sheets in an Excel file.
    """
    excel_filepath = os.path.join(output_dir, filename)
    base_name, _ = os.path.splitext(filename)
    numpy_filepath = os.path.join(output_dir, f"{base_name}_baselines.npy")

    # Prepare Excel data
    config_list, tuned_list, threshold_list = [], [], []
    for exercise, config in config_data.items(): 
        logic = config.get('trajectory_logic', {}) 
        config_list.append({'Exercise': exercise, 'Metric': logic.get('metric'), 'Joints': ', '.join(logic.get('joints', [])), 'Invert_for_Valley': logic.get('invert_for_valley')})
    for exercise, angle_dict in tuned_params_data.items():
         for angle, params in angle_dict.items(): 
             tuned_list.append({'Exercise': exercise, 'Angle': angle, 'Tuned_Prominence': params[0] if params else None, 'Tuned_Distance': params[1] if params else None})
    for exercise, angle_dict in threshold_data.items():
         for angle, threshold_val in angle_dict.items(): 
             threshold_list.append({'Exercise': exercise, 'Angle': angle, 'Derived_Threshold': threshold_val})

    df_config = pd.DataFrame(config_list)
    df_tuned = pd.DataFrame(tuned_list)
    df_thresholds = pd.DataFrame(threshold_list)

    with pd.ExcelWriter(excel_filepath, engine='openpyxl') as writer:
        df_config.to_excel(writer, sheet_name='Exercise_Config', index=False)
        df_tuned.to_excel(writer, sheet_name='Tuned_Counting_Params', index=False)
        df_thresholds.to_excel(writer, sheet_name='Derived_Thresholds', index=False)
    print(f"\n Exported data to file: {filename}...")

    np.save(numpy_filepath, averaged_baseline_data, allow_pickle=True)
    print(f" Averaged baselines exported to: {numpy_filepath}")
    

In [25]:
def load_precomputed_data_from_excel(base_filename, input_dir="."):
    """
    Loads config, params, thresholds from Excel AND baselines from NumPy.)
    """
    print(f"\nLoad data from file: {base_filename}")
    excel_filepath = os.path.join(input_dir, base_filename)
    base_name, _ = os.path.splitext(base_filename)
    numpy_filepath = os.path.join(input_dir, f"{base_name}_baselines.npy")

    loaded_avg_baselines = {}
    loaded_tuned_params = {}
    loaded_derived_thresholds = {}

    df_tuned = pd.read_excel(excel_filepath, sheet_name='Tuned_Counting_Params')
    for _, row in df_tuned.iterrows():
        ex, ang = row['Exercise'], row['Angle']
        prom = float(row['Tuned_Prominence'])
        dist = int(row['Tuned_Distance'])
        
        if ex not in loaded_tuned_params: 
            loaded_tuned_params[ex] = {}
        
        loaded_tuned_params[ex][ang] = (prom if pd.notna(prom) else Default_Peak_Prominence,
                                        dist if pd.notna(dist) else Default_Peak_Distance)

    df_thresh = pd.read_excel(excel_filepath, sheet_name='Derived_Thresholds')
    for _, row in df_thresh.iterrows():
        ex, ang = row['Exercise'], row['Angle']
        thresh_val = None
        try:
             thresh_val = float(row['Derived_Threshold'])
             if np.isnan(thresh_val): thresh_val = None
        except (TypeError, ValueError): 
            pass
        
        if ex not in loaded_derived_thresholds: 
            loaded_derived_thresholds[ex] = {}
        loaded_derived_thresholds[ex][ang] = thresh_val

    loaded_avg_baselines = np.load(numpy_filepath, allow_pickle=True).item()
    print(f"Loaded averaged baselines from: {numpy_filepath}")

    return loaded_avg_baselines, loaded_tuned_params, loaded_derived_thresholds


In [26]:
# In case of re-running the cell
averaged_baselines = {}
tuned_params = {}
derived_thresholds = {}

if Precomputed_model_parameters: 
    averaged_baselines, tuned_params, derived_thresholds = load_precomputed_data_from_excel(Precomputed_model_parameters)
else: 
    averaged_baselines, tuned_params, derived_thresholds = load_baseline_data(
        Video_baseline_path,
        HRNET_MODEL,
        Jointnames_list,
        Target_rep_length,
        EXERCISE_CONFIG,
        Tuning_Prominance_range,
        Tuning_Distance_range,
        threshold_std_multiplier=Threshold_STD_Multiplier,
        min_reps_for_threshold=Min_REPS_to_calc_threshold)

if averaged_baselines:
    learned_data_to_excel(
    config_data=EXERCISE_CONFIG,
    tuned_params_data=tuned_params,
    threshold_data=derived_thresholds,
    averaged_baseline_data=averaged_baselines, 
    filename=Precomputed_model_parameters)
    print("\n Calculated recognition thresholds (Per Angle and Overall)")
else:
    print("\n Baseline Data Loading/Processing Failed👎")


Load data from file: exercise_learned_summary.xlsx
Loaded averaged baselines from: .\exercise_learned_summary_baselines.npy

 Exporting learned data to file: exercise_learned_summary.xlsx...
  Metadata exported to: .\exercise_learned_summary.xlsx
 Averaged baselines exported to: .\exercise_learned_summary_baselines.npy

 Calculated recognition thresholds (Per Angle and Overall)


### Testvideo

This cell analyzing a new video file.

**User Inputs in this Cell:**
* `video_to_test`: (`str` or `pathlib.Path`)
    * Specifies the full path to the video file to be classified. Defaults to `Video_test_path` from Cell 2 if not changed.
* `manual_angle_hint`: (`str` or `None`)
    * Option allows you to tell the recognition function the expected camera angle ('left', 'right', 'front').
    * Unit = Category label (`'left'`, `'right'`, `'front'`) or `None`.

In [ ]:
#DTW_Distance_threshold = 7000

In [27]:
video_to_test = Video_test_path

# Optional angle hint
manual_angle_hint = None #'left', 'right', 'front'

#if proceed_to_recognition:
print("==========================================================================================================================================")
print(f" Recognizing Exercise in Video")
print(f" Video: {os.path.basename(video_to_test)} ")
print("==========================================================================================================================================")

list_of_rep_results, total_reps_detected = recognize_exercise(
    new_video_path=video_to_test,
    averaged_baseline_data=averaged_baselines,
    hrnet_model=HRNET_MODEL,
    joint_names=Jointnames_list,
    exercise_config=EXERCISE_CONFIG,
    Target_rep_length=Target_rep_length,
    tuned_counting_params=tuned_params,
    exercise_specific_thresholds=derived_thresholds,
    hint_angle=manual_angle_hint,
    fallback_threshold=DTW_Distance_threshold)

print("\n==========================================================================================================================================")
print(f" Results for {os.path.basename(video_to_test)}")
print("==========================================================================================================================================")
print(f" Detected {total_reps_detected} Repetitions.")

if not list_of_rep_results:
    print("No valid repetition results to display")
else:
    print("\n Per repetition Analysis")
    match_counts = {}
    successful_reps = 0
    overall_best_label = None
    valid_matches = [res[1] for res in list_of_rep_results if isinstance(res[1], str) and not res[1].startswith("Error") and not res[1].startswith("No Match")]
    if valid_matches:
         temp_match_counts = {}
         for label in valid_matches: 
             temp_match_counts[label] = temp_match_counts.get(label, 0) + 1
         if temp_match_counts: 
             overall_best_label = max(temp_match_counts, key=temp_match_counts.get)

    for rep_index, matched_label, min_distance in list_of_rep_results:
        status = ""
        if isinstance(matched_label, str) and matched_label.startswith("Error"):
             status = f"-> {matched_label}"
        elif isinstance(matched_label, str) and matched_label.startswith("No Match"):
             status = f"-> {matched_label}"
        else:
            status = f"-> Match: {matched_label} (Dist: {min_distance:.2f})"
            match_counts[matched_label] = match_counts.get(matched_label, 0) + 1
            successful_reps += 1
        print(f" Rep {rep_index}: {status}")

    print("\n Overall Summary")
    if successful_reps > 0:
         if match_counts:
             majority_label = max(match_counts, key=match_counts.get)
             majority_count = match_counts[majority_label]
             print(f"Overall match: {majority_label} ({majority_count}/{successful_reps} successfully matched reps)")
         else:
              print(" No specific exercises were matched consistently.")
         print(f" Total reps successfully matched: {successful_reps} / {total_reps_detected}")
    elif total_reps_detected > 0:
         print("No repetitions were successfully matched to any baseline exercise.")
    else:
         print("No repetitions were detected in the video")
print("==========================================================================================================================================\n")

 Recognizing Exercise in Video
 Video: test_excercise.mp4 

 Recognizing Exercise (Angle Hint='None') for: test_excercise.mp4 ---
Detected 5 potential repetitions in test video.
Comparing each detected repetition against 'overall' baselines...

 Results for test_excercise.mp4
 Detected 5 Repetitions.

 Per repetition Analysis
 Rep 1: -> Match: leg_extension (Dist: 844.21)
 Rep 2: -> No Match (Closest: leg_extension, Dist: 1224.46 >= 1049.06)
 Rep 3: -> Match: tricep_pushdown (Dist: 1529.70)
 Rep 4: -> Match: tricep_pushdown (Dist: 1911.55)
 Rep 5: -> No Match (Closest: leg_extension, Dist: 1532.61 >= 1049.06)

 Overall Summary
Overall match: tricep_pushdown (2/3 successfully matched reps)
 Total reps successfully matched: 3 / 5

